In [4]:
import yfinance as yahooFinance
import pandas as pd
import csv
import os
# import yfinance as yahooFinance
import pandas as pd
import csv
from pathlib import Path
import numpy as np
from scipy.stats import norm

In [13]:
stocks_list = ["GOOG","AAPL"]

In [14]:
def get_data(stock):
    data = pd.DataFrame()
    data = yahooFinance.Ticker(stock).history(period="1y")['Close']
    return data

In [15]:
data = []
for i in range(len(stocks_list)):
    data.append(get_data(stocks_list[i]))
    data[i].to_csv(f"yahoo_data/{i}.csv")

In [5]:


# stocks_list = ["GOOG"]

# def get_data(stock):
#     data = pd.DataFrame()
#     data = yahooFinance.Ticker(stock).history(period="1y")['Close']
#     return data

# data = []
# for i in range(len(stocks_list)):
#     data.append(get_data(stocks_list[i]))

data = []
# directory = '/home/maths/dual/mt6200886/project/yahoo_data_' + str(stocks)
directory = '/Users/prishajain/Desktop/MTD350/yahoo_data/to_convert'
 
for filename in os.listdir(directory):
    temp_data = []
    with open(directory + '/' + filename, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        for row in csv_reader:
            date = row[0]
            close_price = float(row[1])
            temp_data.append((date, close_price))

    series_data = pd.Series([x[1] for x in temp_data], index=[x[0] for x in temp_data])
    data.append(series_data)

def calculate_prices(data,days,trials):
    log_return = np.log(1 + data.pct_change()) # periodic daily return = ln(today's price/previous day's price)
    u = log_return.mean() 
    var = log_return.var()
    drift = u - (0.5*var) # drift, used for direction
    stdev = log_return.std()
    Z = norm.ppf(np.random.rand(days, trials))
    daily_returns = np.exp(drift + stdev * Z)
    price_paths = np.zeros_like(daily_returns)
    price_paths[0] = data.iloc[-1]
    for t in range(1, days):
        price_paths[t] = price_paths[t-1]*daily_returns[t]     
    return price_paths

def strategy_initialise():
    
    # using Monte Carlo simulation 
    days = 100+1
    trials = 10000
    mean = np.zeros(days)
    sigma = np.zeros(days)
    prices = calculate_prices(data[0],days,trials)
    for j in range(days):
        mean[j] = np.mean(prices[j])
        sigma[j] = np.std(prices[j])
    np.save('mean&sigma.npy', [mean, sigma])

strategy_initialise()
        
    

In [6]:
data[0]

2023-01-03     89.830002
2023-01-04     91.010002
2023-01-05     88.070000
2023-01-06     87.360001
2023-01-09     89.195000
                 ...    
2023-12-22    142.130005
2023-12-26    142.979996
2023-12-27    142.830002
2023-12-28    141.850006
2023-12-29    140.679993
Length: 250, dtype: float64

In [7]:
data[0]

(250,)

In [8]:
type(data[0])

pandas.core.series.Series